In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time

In [2]:
def ETL(state):
    zillow_df = pd.read_csv('../Resources2/Zillow_Sale_Prices_Zip.csv')
    zillow_df.set_index('RegionName', inplace=True)
    state_df = zillow_df.loc[zillow_df.StateName==state,:]
    state_df = state_df.dropna()
    median09 = state_df.loc[:,'2009-01':'2009-12'].median(axis = 1, skipna = True)
    median19 = state_df.loc[:,'2019-01':'2019-12'].median(axis = 1, skipna = True)
    new_df=pd.concat([median09, median19], axis=1)
    new_name= ['Median Price 2009','Median Price 2019']
    new_df.columns=new_name
    new_df['Inflated Median Price 2009'] = 1.1917 * new_df['Median Price 2009']
    new_df['Percent Change'] = (new_df['Median Price 2019'] - new_df['Inflated Median Price 2009'])/new_df['Inflated Median Price 2009']
    new_df = new_df[['Median Price 2009','Inflated Median Price 2009','Median Price 2019','Percent Change']]
    new_df['Median Price 2009'] = new_df['Median Price 2009'].map("${:,.2f}".format)
    new_df['Inflated Median Price 2009'] = new_df['Inflated Median Price 2009'].map("${:,.2f}".format)
    new_df['Median Price 2019'] = new_df['Median Price 2019'].map("${:,.2f}".format)
    new_df['Percent Change'] = new_df['Percent Change'].map("{:.2%}".format)
    
    return new_df.head()

In [3]:
ETL('Arizona')

,Median Price 2009,Inflated Median Price 2009,Median Price 2019,Percent Change
RegionName,,,,
85022,"$141,900.00","$169,102.23","$237,800.00",40.62%
85023,"$112,100.00","$133,589.57","$239,150.00",79.02%
85024,"$170,700.00","$203,423.19","$265,800.00",30.66%
85027,"$97,950.00","$116,727.01","$213,450.00",82.86%
85032,"$122,950.00","$146,519.51","$238,200.00",62.57%
